<a href="https://colab.research.google.com/github/Signed-B/build-your-own-llm/blob/main/lecture_workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How To Build a Large Language Model
### Presented by Beckett Hyde - 03/05/2024

Welcome to this notebook that walks you through all the steps necessary for creating your first LLM application:
+ Pulling pre-trained base model weights
+ Loading those weights into memory, incuding on small, consumer GPUs
+ Pulling and tokenizing a dataset (or tokenizing your own dataset)
+ Fine-tuning the model
+ Model inference

This is designed to go along with the in-person presentation given on March 5th to the University of Colorado, Boulder. This does not cover training a model from scratch (as that is extremely time and resource intensive) and takes extra steps to ensure the model can even work on higher-end consumer hardware (no NVIDIA A100 GPUs required! If your machine has ~15GB of vRAM, sometimes even less, this could work off of colab's T4s for you).

If you are interested in skipping straight to the working solutions, see [this notebook](example.com). If you are interested in the theory portion of the presetation, see [this sldieshow](example.com).

<hr />

This presentation was sponsored by the CU Boulder Undergraduate SIAM Chapter. A thank you to them for their support in promoting and financing the event.

<center>
<p float="left">
  <img src="https://www.colorado.edu/brand/sites/default/files/styles/medium/public/block/boulder-one-line_4.png" width="300" />
  <img width="10" hspace="10" />
  <img src="https://www.siam.org/portals/0/Logo%20Guide/logo_cobrand.png" width="300" />
</p>
</center>
<hr/>
<hr />

# Step 1: Install Software & Download Weights

A number of packages, some with specific versions, are required for this notebook to work (especially with the modifications we make for smaller hardware). For simplicity, we give these to you. Run the below cell to prepare your environment.

### A note about Google Colab:

Ensure you are running on Colab's T4 environment, which is available as part of the free tier.

It is also possibe that you may have to disconnect and reconnect to your environment for some of these installs to take effect (particularly `accelerate`).

In [1]:
!pip install -q transformers==4.34.0
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets einops

model_id = "eluzhnica/mpt-7b-8k-peft-compatible"

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# Config for 4-bit downloading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0}
                                             )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

# Step 2: Load and tokenize our data

Huggingface, like Kaggle, holds publicly available datasets we can use, like `vicgalle/alpaca-gpt4`, which was actually created, [ironically](https://en.wikipedia.org/wiki/Dead_Internet_theory), using GPT-4!

Each model comes with a pre-trained tokenizer we can use. This converts the text into a series of tokens represented in a high-dimensional space where relative proximity encodes meaning. We use the `datasets` library to manage our dataseets and the `transformers.AutoTokenizer` class to pull and hold our tokenizer.


Almost all interesting pre-trained models (both base-models are available on [Huggingface](huggingface.co). We will be downloading a specific version of the `MPT-7B` model, released in July 2018, with modifications to work on our limited hardware, and using its tokenizer.

The model is `eluzhnica/mpt-7b-8k-peft-compatible` available [here](https://huggingface.co/eluzhnica/mpt-7b-8k-peft-compatible).

### Let's explore the data!

You aren't like those *other* data scientists and MLEs, you actually *do your job!* (please god I am done fixing your messes).

### Train test splits

# Step 3: Download & configure the base model



In order for our model to work, we need to use gradient checkpointing and "reformat" the model to work with our artificially reduced bit-sizes.

How big is our model? How many parameters are trainable?

In [2]:
# print("All params:", sum([param.numel() for _, param in model.named_parameters()]))
# print("Trainable params:", sum([param.numel() if param.requires_grad else 0 for _, param in model.named_parameters()]))

# Step 4: Prepare our model for training & inference

Our model currently has no trainable paramaters. We're going to use LoRA as a training algorithm (using `peft`) with a lot of hyperparameters to control how we train.


How many parameters are trainable now?

In [3]:
# print("All params:", sum([param.numel() for _, param in model.named_parameters()]))
# print("Trainable params:", sum([param.numel() if param.requires_grad else 0 for _, param in model.named_parameters()]))

# Step 4.5: Test the base model

Warning: the model is completely untrained and simply predicts the most likely next token. It is very easy to get it to "say" unsavory things at this stage.

Let's try it with one of our prompts.

# Step 5: Train the model!

Using the full power of `transformers` now, we create a training plan with `TrainingArguments` and a trainer with `Trainer`.

We use the following settings:
TODO

Now we train!

# Step 6: Inference

Now we use the model to answer some questions.

In [4]:
def stream(question, context=None):
    system_prompt = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n'
    inst_tag, input_tag, resp_tag = "### Instruction:\n", "### Input:\n", "### Response:\n"

    prompt = f"{system_prompt}{inst_tag}{question.strip()}\n\n{input_tag}{context.strip()}\n\n{resp_tag}" \
             if context else f"{system_prompt}{inst_tag}{question.strip()}\n\n{resp_tag}"

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Despite returning the usual output, the streamer will also print the generated text to stdout.
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=50)

## There you go!

This is a rudimentary, but very effective, question-answering language model.